In [ ]:
#default_exp data.seq2seq.summarization

In [ ]:
#all_slow

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# data.seq2seq.summarization

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for summarization tasks using architectures like BART and T5.

In [ ]:
#export
import ast
from functools import reduce

import torch
from transformers import *
from fastai.text.all import *

from blurr.utils import *
from blurr.data.core import *
from blurr.data.seq2seq.core import *

logging.set_verbosity_error()

In [ ]:
#hide
import pdb

from nbdev.showdoc import *
from fastcore.test import *

from fastai import __version__ as fa_version
from torch import __version__ as pt_version
from transformers import __version__ as hft_version

print(f'Using pytorch {pt_version}')
print(f'Using fastai {fa_version}')
print(f'Using transformers {hft_version}')

Using pytorch 1.7.1
Using fastai 2.4
Using transformers 4.8.1


In [ ]:
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Summarization tokenization, batch transform, and DataBlock methods

Summarization tasks attempt to generate a human-understandable and sensible representation of a larger body of text (e.g., capture the meaning of a larger document in 1-3 sentences).

In [ ]:
path = Path('./')
cnndm_df = pd.read_csv(path/'cnndm_sample.csv'); len(cnndm_df)

1000

In [ ]:
cnndm_df.head(2)

,article,highlights,ds_type
0,"(CNN) -- Globalization washes like a flood over the world's cultures and economies. Floods can be destructive; however, they can also bring blessings, as the annual floods of the Nile did for ancient Egypt. The world's great universities can be crucial instruments in shaping, in a positive way, humankind's reaction to globalization and the development of humankind itself. Traditionally, universities have been defined and limited by location, creating an academic community and drawing students and scholars to that place. Eventually, some universities began to encourage students to study el...","John Sexton: Traditionally, universities have been defined and limited by location .\nGlobal campuses form a network of thought, innovation, he writes .\nFaculty can teach, Sexton says, students can team up in many cities at once .\nSexton: Research, scholarship can be shared and cultural ties made in ""century of knowledge""",train
1,"(CNN) -- Armenian President Robert Kocharian declared a state of emergency Saturday night after a day of clashes between police and protesters, a spokeswoman for the Armenian Foreign Ministry said. Opposition supporters wave an Armenian flag during a protest rally in Yerevan, Armenia, on Saturday. The protesters claim last month's presidential election was rigged. The state of emergency will ""hopefully bring some order"" to the capital, Yerevan, said Salpi Ghazarian, assistant to the Armenian foreign minister, who spoke to CNN early Sunday. The state of emergency could last until March 20, ...","NEW: Protest moves after crackdown at Freedom Square .\nOrder sought after protests over last month's election turn violent .\nDemonstrators say the election was fraudulent .\nState of emergency could last until March 20, official says .",train


In [ ]:
pretrained_model_name = "facebook/bart-large-cnn"
model_cls = AutoModelForSeq2SeqLM

hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)
hf_arch, type(hf_tokenizer), type(hf_config), type(hf_model)

('bart',
 transformers.models.bart.tokenization_bart_fast.BartTokenizerFast,
 transformers.models.bart.configuration_bart.BartConfig,
 transformers.models.bart.modeling_bart.BartForConditionalGeneration)

In [ ]:
blocks = (HF_Seq2SeqBlock(hf_arch, hf_config, hf_tokenizer, hf_model), noop)

dblock = DataBlock(blocks=blocks, 
                   get_x=ColReader('article'), 
                   get_y=ColReader('highlights'), 
                   splitter=RandomSplitter())

Two lines!  Notice we pass in `noop` for our targets (e.g. our summaries) because the batch transform will take care of both out inputs and targets.

In [ ]:
# dblock.summary(cnndm_df)

In [ ]:
dls = dblock.dataloaders(cnndm_df, bs=4)

In [ ]:
b = dls.one_batch()

In [ ]:
len(b), b[0]['input_ids'].shape, b[0]['labels'].shape, b[1].shape

(2, torch.Size([4, 1024]), torch.Size([4, 74]), torch.Size([4, 74]))

In [ ]:
dls.show_batch(dataloaders=dls, max_n=2, input_trunc_at=1000, target_trunc_at=250)

,text,target
0,"<s> (CNN) -- Home to up to 10 percent of all known species, Mexico is recognized as one of the most biodiverse regions on the planet. The twin threats of climate change and human encroachment on natural environments are, however, threatening the existence of the country's rich wildlife. And there is a great deal to lose. In the United Nations Environment Program (UNEP) World Conservation Monitoring Centre's list of megadiverse countries Mexico ranks 11th. The list represents a group of 17 countries that harbor the majority of the Earth's species and are therefore considered extremely biodiverse. From its coral reefs in the Caribbean Sea to its tropical jungles in Chiapas and the Yucatan peninsula and its deserts and prairies in the north, Mexico boasts an incredibly rich variety of flora and fauna. Some 574 out of 717 reptile species found in Mexico -- the most in any country -- can only be encountered within its borders. It is home to 502 types of mammals, 290 species of birds, 1,150","Mexico hosts to up to 10 percent of all known species on Earth.\nIt is home to 502 types of mammals, 290 bird species and 26,000 types of plants.\nHuman development and climate change is placing a big strain on its biodiversity.\nThe Golden Eagle is un"
1,"<s> Washington (CNN)Almost immediately following the news of the first terrorist attacks that eventually killed 17 people across France, the global community united around a Twitter hashtag ""Je suis Charlie"" and just days later foreign leaders linked arms with their French counterparts to lead a historic million-person strong rally. Meanwhile, explosives strapped to a girl who appeared to be about 10-years-old detonated on Saturday, killing at least 20 people, in a country whose encounters with terrorism were also punctuated by a hashtag -- this time ""#BringBackOurGirls"" of Nigeria. Boko Haram militants killed as many as 2,000 people, mostly civilians,in a massacre that started the weekend before the terror attack on Charlie Hedbo in downtown Paris. Both the attacks in Nigeria and those in Paris are shocking and horrifying in their own respects, and yet one fomented an unprecedented international reaction -- a popular show of force that rivaled even the reaction to 9/11 -- while the re","France and Nigeria experienced waves of terrorism during the first weeks of 2015.\nWhile the terror attacks in Paris sparked international unified outrage, reaction to Nigeria was more muted.\nSymbolism, politics and media all played a role in how Fra"


## Tests

The purpose of the following tests is to ensure as much as possible, that the core DataBlock code above works for the pretrained **summarization models** below.  These tests are excluded from the CI workflow because of how long they would take to run and the amount of data that would be required to download.

**Note**: Feel free to modify the code below to test whatever pretrained summarization models you are working with ... and if any of your pretrained summarization models fail, please submit a github issue *(or a PR if you'd like to fix it yourself)*

In [ ]:
[ model_type for model_type in BLURR.get_models(task='ConditionalGeneration') 
 if (not model_type.__name__.startswith('TF')) ]

[transformers.models.bart.modeling_bart.BartForConditionalGeneration,
 transformers.models.bigbird_pegasus.modeling_bigbird_pegasus.BigBirdPegasusForConditionalGeneration,
 transformers.models.blenderbot.modeling_blenderbot.BlenderbotForConditionalGeneration,
 transformers.models.blenderbot_small.modeling_blenderbot_small.BlenderbotSmallForConditionalGeneration,
 transformers.models.fsmt.modeling_fsmt.FSMTForConditionalGeneration,
 transformers.models.led.modeling_led.LEDForConditionalGeneration,
 transformers.models.m2m_100.modeling_m2m_100.M2M100ForConditionalGeneration,
 transformers.models.mbart.modeling_mbart.MBartForConditionalGeneration,
 transformers.models.mt5.modeling_mt5.MT5ForConditionalGeneration,
 transformers.models.pegasus.modeling_pegasus.PegasusForConditionalGeneration,
 transformers.models.prophetnet.modeling_prophetnet.ProphetNetForConditionalGeneration,
 transformers.models.speech_to_text.modeling_speech_to_text.Speech2TextForConditionalGeneration,
 transformers.mo

In [ ]:
pretrained_model_names = [
    'facebook/bart-base',
    'facebook/blenderbot_small-90M',
    'allenai/led-base-16384',
    'google/mt5-small',
    'google/pegasus-cnn_dailymail',
    't5-small', 
    'microsoft/prophetnet-large-uncased',
    'microsoft/xprophetnet-large-wiki100-cased', # XLMProphetNet
]

In [ ]:
path = Path('./')
cnndm_df = pd.read_csv(path/'cnndm_sample.csv')

In [ ]:
#slow
#hide_output
model_cls = AutoModelForSeq2SeqLM
bsz = 2
seq_sz = 256
trg_seq_sz = 40

test_results = []
for model_name in pretrained_model_names:
    error=None
    
    print(f'=== {model_name} ===\n')
    
    hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(model_name, model_cls=model_cls)
    print(f'architecture:\t{hf_arch}\ntokenizer:\t{type(hf_tokenizer).__name__}\n')
    
    # not all architectures include a native pad_token (e.g., gpt2, ctrl, etc...), so we add one here
    if (hf_tokenizer.pad_token is None): 
        hf_tokenizer.add_special_tokens({'pad_token': '<pad>'})  
        hf_config.pad_token_id = hf_tokenizer.get_vocab()['<pad>']
        hf_model.resize_token_embeddings(len(hf_tokenizer))   
    
    before_batch_tfm = HF_Seq2SeqBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model,
                                                      padding='max_length', 
                                                      max_length=seq_sz, 
                                                      max_target_length=trg_seq_sz)
    
    def add_t5_prefix(inp): return f'summarize: {inp}' if (hf_arch == 't5') else inp
    
    blocks = (HF_Seq2SeqBlock(before_batch_tfm=before_batch_tfm), noop)
    dblock = DataBlock(blocks=blocks, 
                   get_x=Pipeline([ColReader('article'), add_t5_prefix]), 
                   get_y=ColReader('highlights'), 
                   splitter=RandomSplitter())

    dls = dblock.dataloaders(cnndm_df, bs=bsz) 
    b = dls.one_batch()

    try:
        print('*** TESTING DataLoaders ***\n')
        test_eq(len(b), 2)
        test_eq(len(b[0]['input_ids']), bsz)
        test_eq(b[0]['input_ids'].shape, torch.Size([bsz, seq_sz]))
        test_eq(len(b[1]), bsz)
        test_eq(b[1].shape, torch.Size([bsz, trg_seq_sz]))

        if (hasattr(hf_tokenizer, 'add_prefix_space') and hf_arch not in ['led']):
             test_eq(hf_tokenizer.add_prefix_space, True)
            
        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, 'PASSED', ''))
        dls.show_batch(dataloaders=dls, max_n=2, input_trunc_at=1000)
        
    except Exception as err:
        test_results.append((hf_arch, type(hf_tokenizer).__name__, model_name, 'FAILED', err))

=== facebook/bart-base ===

architecture:	bart
tokenizer:	BartTokenizerFast

*** TESTING DataLoaders ***



,text,target
0,"<s> (CNN) -- Home to up to 10 percent of all known species, Mexico is recognized as one of the most biodiverse regions on the planet. The twin threats of climate change and human encroachment on natural environments are, however, threatening the existence of the country's rich wildlife. And there is a great deal to lose. In the United Nations Environment Program (UNEP) World Conservation Monitoring Centre's list of megadiverse countries Mexico ranks 11th. The list represents a group of 17 countries that harbor the majority of the Earth's species and are therefore considered extremely biodiverse. From its coral reefs in the Caribbean Sea to its tropical jungles in Chiapas and the Yucatan peninsula and its deserts and prairies in the north, Mexico boasts an incredibly rich variety of flora and fauna. Some 574 out of 717 reptile species found in Mexico -- the most in any country -- can only be encountered within its borders. It is home to 502 types of mammals, 290 species of birds, 1,150","Mexico hosts to up to 10 percent of all known species on Earth.\nIt is home to 502 types of mammals, 290 bird species and 26,000 types of plants.\nHuman development"
1,"<s> (CNN) -- Two weeks. Two gut-wrenching, frustrating, mysterious weeks. That's how long it's been since 227 passengers and 12 crew members boarded Malaysia Airlines Flight 370, destined for Beijing. A routine trip, it seemed, to catch up relatives in time for the weekend, start on a work assignment or just get away. Where they got to, still unknown. An exhaustive search -- covering a mind-boggling 2.97 million square miles, which is nearly the size of the continental United States -- has yielded some clues, but no proof of where the Boeing 777 is or definitively what happened to it. The latest, most notable lead revolved around two large objects detected by satellite Sunday floating on waters over 1,400 miles off of Australia's west coast. The first of several Australian military planes, as well as two long-range commercial jets, resumed their search Saturday morning to find any trace of the objects, amid some skepticism that they or ships in the area ever will and, if they do, that",NEW: Planes depart Australia to resume their search for airplane debris.\nNEW: Official: Passengers' relatives are moved to a different Kuala Lumpur hotel.\nObjects seen on satellite spark


=== facebook/blenderbot_small-90M ===

architecture:	blenderbot_small
tokenizer:	BlenderbotSmallTokenizer

*** TESTING DataLoaders ***



,text,target
0,"__unk__ cnn ) __unk__ - home to up to 10 percent of all known species, mexico is recognized as one of the most biodiverse regions on the planet. the twin threats of climate change and human encroachment on natural environments are, however, threatening the existence of the country's rich wildlife. and there is a great deal to lose. in the united nations environment program __unk__ unep ) world conservation monitoring centre's list of megadiverse countries mexico ranks 11th. the list represents a group of 17 countries that harbor the majority of the earth's species and are therefore considered extremely biodiverse. from its coral reefs in the caribbean sea to its tropical jungles in chiapas and the yucatan peninsula and its de__unk__ and prairies in the north, mexico boasts an incredibly rich variety of flora and fauna. some 574 out of 717 reptile species found in mexico __unk__ - the most in any country __unk__ - can only be encountered within its borders. it is home to 502 types of ma","mexico hosts to up to 10 percent of all known species on earth. __newln__ it is home to 502 types of mammals, 290 bird species and 26 000 types of plants. __newln__ human development"
1,"__unk__ cnn ) __unk__ - it's a congested, sprawling transport hub surrounded by 1950s architecture and predominantly used by commuters or tourists to cross the city of istanbul. but proposed changes to taksim square have seen it become the flashpoint for protests that have swept through turkey in the past week, leaving thousands injured and focusing the world's attention on the government of prime minister recep tayyip erdogan. taksim has been no stranger to violence. in 1977, at least 34 protesters died during may day clashes with police. may 1 rallies in the square were banned in 1980 and were only allowed to legally resume in 2010. on may day this year, there were riots after city authorities again refused to grant trade unions and youth groups permission to demonstrate in taksim, blaming construction work being carried out in the square. professor ersin kalaycioglu, professor of political science at istanbul's sabanci university, said significantly, taksim square was also known as",taksim square was where istanbul's water was distributed - taksim means divide. __newln__ the site is seen as symbolizing the seclar turkish republic founded by atk.


=== allenai/led-base-16384 ===

architecture:	led
tokenizer:	LEDTokenizerFast

*** TESTING DataLoaders ***



...

,text,target
0,"summarize: Dan Condon believes in recycling. Just not when it comes to his hotel towels. Condon composts when he's at home in Boulder, Colorado. He eats local, organic and fair-trade food and drives a Honda CR-Z hybrid sports car. You might call him green. Except he's not so green when he travels for his work at an education nonprofit and stays in a hotel, which happens about 10 weeks per year. There, he uses a new towel every day. And don't try to bribe him with a drink or dessert coupon to get him to reuse the same one. ""I could care less about rewards for environmentally conscious behavior unless it's miles,"" Condon wrote in an e-mail. If hotels can't convince a hybrid-driving recycling enthusiast like Condon to go green while traveling, how can they possibly convince everyone else? 9 glamorous movie-star hotels. That's the problem of hotels trying to ""green"" your hotel stay. After guests have paid a pretty penny for a night at the inn, even the most environmental guests may want to","Hotel guests who ""go green"" are happier with their stay. Increasing water and energy costs are pushing hotels to cut costs wherever they can. Many hotels find that guests don't"
1,"summarize: Some U.S. officials this year are expected to get smartphones capable of handling classified government documents over cellular networks, according to people involved in the project. The phones will run a modified version of Google's Android software, which is being developed as part of an initiative that spans multiple federal agencies and government contractors, these people said. The smartphones are first being deployed to U.S. soldiers, people familiar with the project said. Later, federal agencies are expected to get phones for sending and receiving government cables while away from their offices, sources said. Eventually, local governments and corporations could give workers phones with similar software. The Army has been testing touchscreen devices at U.S. bases for nearly two years, said Michael McCarthy, a director for the Army's Brigade Modernization Command, in a phone interview. About 40 phones were sent to fighters overseas a year ago, and the Army plans to ship","Government, military officials to get Android phones capable of sharing secret documents. The phones will run a modified version of Google's Android software, sources say. Contractor: Google ""more"


=== microsoft/prophetnet-large-uncased ===

architecture:	prophetnet
tokenizer:	ProphetNetTokenizer

*** TESTING DataLoaders ***



,text,target
0,"( cnn ) - - home to up to 10 percent of all known species, mexico is recognized as one of the most biodiverse regions on the planet. the twin threats of climate change and human encroachment on natural environments are, however, threatening the existence of the country's rich wildlife. and there is a great deal to lose. in the united nations environment program ( unep ) world conservation monitoring centre's list of megadiverse countries mexico ranks 11th. the list represents a group of 17 countries that harbor the majority of the earth's species and are therefore considered extremely biodiverse. from its coral reefs in the caribbean sea to its tropical jungles in chiapas and the yucatan peninsula and its deserts and prairies in the north, mexico boasts an incredibly rich variety of flora and fauna. some 574 out of 717 reptile species found in mexico - - the most in any country - - can only be encountered within its borders. it is home to 502 types of mammals, 290 species of birds, 1,","mexico hosts to up to 10 percent of all known species on earth. it is home to 502 types of mammals, 290 bird species and 26, 000 types of plants. human development and climate"
1,"some u. s. officials this year are expected to get smartphones capable of handling classified government documents over cellular networks, according to people involved in the project. the phones will run a modified version of google's android software, which is being developed as part of an initiative that spans multiple federal agencies and government contractors, these people said. the smartphones are first being deployed to u. s. soldiers, people familiar with the project said. later, federal agencies are expected to get phones for sending and receiving government cables while away from their offices, sources said. eventually, local governments and corporations could give workers phones with similar software. the army has been testing touchscreen devices at u. s. bases for nearly two years, said michael mccarthy, a director for the army's brigade modernization command, in a phone interview. about 40 phones were sent to fighters overseas a year ago, and the army plans to ship 50 more","government, military officials to get android phones capable of sharing secret documents. the phones will run a modified version of google's android software, sources say. contractor : google "" more cooperative "" than"


=== microsoft/xprophetnet-large-wiki100-cased ===

architecture:	xlm_prophetnet
tokenizer:	XLMProphetNetTokenizer

*** TESTING DataLoaders ***



,text,target
0,"(CNN) -- Home to up to 10 percent of all known species, Mexico is recognized as one of the most biodiverse regions on the planet. The twin threats of climate change and human encroachment on natural environments are, however, threatening the existence of the country's rich wildlife. And there is a great deal to lose. In the United Nations Environment Program (UNEP) World Conservation Monitoring Centre's list of megadiverse countries Mexico ranks 11th. The list represents a group of 17 countries that harbor the majority of the Earth's species and are therefore considered extremely biodiverse. From its coral reefs in the Caribbean Sea to its tropical jungles in Chiapas and the Yucatan peninsula and its deserts and prairies in the north, Mexico boasts an incredibly rich variety of flora and fauna. Some 574 out of 717 reptile species found in Mexico -- the most in any country -- can only be encountered within its borders. It is home to 502 types of mammals, 290 species of birds, 1,150 vari","Mexico hosts to up to 10 percent of all known species on Earth. It is home to 502 types of mammals, 290 bird species and 26,000 types of plants"
1,"Washington (CNN) -- Moments after the U.S. Supreme Court ruled on a pair of same-sex marriage cases, the handful of Democrats considering running for president in 2016 stumbled over themselves in a rush of celebratory reaction, blasting out a salvo of congratulatory press releases and tweets. Rulings hailed as historic victory. Voting 5-4 in each of two decisions, the justices struck down part of the Defense of Marriage Act that denied federal benefits to same-sex couples and cleared the way for gays and lesbians to once again marry in California. Former Secretary of State Hillary Clinton, whose husband signed DOMA into law in 1996, applauded the decision. ""By overturning the Defense of Marriage Act, the court recognized that discrimination towards any group holds us all back in our efforts to form a more perfect union,"" Clinton said in a joint statement with her husband, Bill Clinton. Maryland Gov. Martin O'Malley called the rulings ""a powerful step forward."" New York Gov. Andrew Cuom","Democrats who support same-sex marriage are now playing on increasingly friendly political turf. GOP leaders express dismay at ruling, but seem eager to be rid of a"


In [ ]:
#slow
#hide_input
test_results_df = pd.DataFrame(test_results, columns=['arch', 'tokenizer', 'model_name', 'result', 'error'])
display_df(test_results_df)

,arch,tokenizer,model_name,result,error
0,bart,BartTokenizerFast,facebook/bart-base,PASSED,
1,blenderbot_small,BlenderbotSmallTokenizer,facebook/blenderbot_small-90M,PASSED,
2,led,LEDTokenizerFast,allenai/led-base-16384,PASSED,
3,mt5,T5TokenizerFast,google/mt5-small,PASSED,
4,pegasus,PegasusTokenizerFast,google/pegasus-cnn_dailymail,PASSED,
5,t5,T5TokenizerFast,t5-small,PASSED,
6,prophetnet,ProphetNetTokenizer,microsoft/prophetnet-large-uncased,PASSED,
7,xlm_prophetnet,XLMProphetNetTokenizer,microsoft/xprophetnet-large-wiki100-cased,PASSED,


## Summary

This module includes the fundamental data preprocessing bits to use Blurr for summarization.

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01_modeling-core.ipynb.
Converted 02_data-language-modeling.ipynb.
Converted 02_modeling-language-modeling.ipynb.
Converted 03_data-token-classification.ipynb.
Converted 03_modeling-token-classification.ipynb.
Converted 04_data-question-answering.ipynb.
Converted 04_modeling-question-answering.ipynb.
Converted 10_data-seq2seq-core.ipynb.
Converted 10_modeling-seq2seq-core.ipynb.
Converted 11_data-seq2seq-summarization.ipynb.
Converted 11_modeling-seq2seq-summarization.ipynb.
Converted 12_data-seq2seq-translation.ipynb.
Converted 12_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-high-level-api.ipynb.
Converted 99b_examples-glue.ipynb.
Converted 99c_examples-glue-plain-pytorch.ipynb.
Converted 99d_examples-multilabel.ipynb.
Converted index.ipynb.
